In [1]:
import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
import sys
import tweepy
import pycountry
import re

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from textblob import TextBlob
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from collections import Counter
from sklearn import model_selection

import warnings
warnings.filterwarnings("ignore")

C:\Users\User\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
data = pd.read_csv('C:/Users/User/tweets_train.csv', encoding="ISO-8859-1")
nltk.download('stopwords')
stop = set(stopwords.words('english')) 
sbs = nltk.stem.SnowballStemmer('english') 
def del_html(text_e): 
    dele1 = re.compile('<.*?>')
    dele2 = re.sub(str(dele1), ' ', str(text_e))
    return dele2
def del_punc(text_e): 
    delete = re.sub(r'[?|!|\'|"|#]',r'',text_e)
    delete = re.sub(r'[.|,|)|(|\|/]',r' ',delete)
    return  delete

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
i=0
origin_text=' '
new_text=[]
posi_text=[] 
neg_text=[] 
sent_num=''
for sent in data['text'].values:
    del_text=[]
    sent=del_html(sent) 
    for w in sent.split():
        for delete_words in del_punc(w).split():
            if((delete_words.isalpha()) & (len(delete_words)>2)):    
                if(delete_words.lower() not in stop):
                    sent_num=(sbs.stem(delete_words.lower())).encode('utf8')
                    del_text.append(sent_num)
                    if (data['sentiment'].values)[i] == "pos": 
                        posi_text.append(sent_num) 
                    if(data['sentiment'].values)[i] == "neg":
                        neg_text.append(sent_num) 
                else:
                    continue
            else:
                continue 
    
    origin_text = b" ".join(del_text) 
    
    new_text.append(origin_text)
    i+=1
    
data['text_split']=new_text
data.shape

(100000, 5)

In [5]:
def ToNum(x):
    if x=="neg":
        return 0
    elif x=="pos":
        return 1
    return x

In [6]:
num_sentiment = data["sentiment"].map(ToNum)
data["sen_num"] = num_sentiment
ttest = data[:25000]
sample = data[25000:50000]
ts1 = sample["text_split"]
sn1 = sample["sen_num"]
ts2 = ttest["text_split"]
sn2 = ttest["sen_num"]
sn1=sn1.astype('int')
sn2=sn2.astype('int')

In [7]:
vec = TfidfVectorizer(ngram_range=(1,2))
vec_s = vec.fit_transform(ts1.values)
vec_t = vec.transform(ts2.values)
val1 = list(np.arange(1,50,5))
len(val1)

10

In [10]:
from sklearn.naive_bayes import MultinomialNB
val_nums=[]
for j in val1:
    sent_ana = MultinomialNB(alpha=j)
    sen_nums = cross_val_score(sent_ana, vec_s, sn1, cv=5, scoring='accuracy')
    val_nums.append(sen_nums.mean())
    
MSE = [1 - x for x in val_nums]
MSE_result = val1[MSE.index(min(MSE))]

In [11]:
sent_ana = MultinomialNB(alpha=6)
sent_ana.fit(vec_s,sn1)

MultinomialNB(alpha=6)

In [12]:
data = pd.read_csv('C:/Users/User/new10k.csv')
newdata=data[["created_at","text","author.location","geo.country","geo.full_name","geo.name"]]
newdata.isnull().sum().sort_values(ascending=False)
newdata=newdata.dropna()
newdata.head()
text10k=[]
for ttext in newdata["text"].apply(str):
    Word_Split = []
    for word in  re.sub("\W"," ",ttext ).split():
        Word_Split.append(word)
    text10k.append(' '.join(Word_Split))
newdata["Word_Split"]= text10k
newdata.head()

created_at  \
1  2022-11-07T23:44:44.000Z   
3  2022-11-07T23:40:17.000Z   
5  2022-11-07T23:15:40.000Z   
7  2022-11-07T22:36:52.000Z   
9  2022-11-07T21:54:10.000Z   

                                                text          author.location  \
1  I am nervous but optimistic Democrats will win...           Huntsville, AL   
3  @joysephstjohn Brilliant interview with Kathy ...     Huntsville, Alabama    
5  @JoeBiden we need another stimulus check $$ ht...           Birmingham, AL   
7  @mark_esque @MrMatthewCFB Very well could be, ...  Mobile, Alabama/ Auburn   
9  Joe Biden cares more about the millions of pot...          Heart of Dixie    

     geo.country   geo.full_name    geo.name  \
1  United States  Huntsville, AL  Huntsville   
3  United States  Huntsville, AL  Huntsville   
5  United States  Birmingham, AL  Birmingham   
7  United States      Daphne, AL      Daphne   
9  United States     Satsuma, AL     Satsuma   

                                          Word_Split  
1  I am nervous but optimistic Democrats will win...  
3  joysephstjohn Brilliant interview with Kathy W...  
5  JoeBiden we need another stimulus check https ...  
7  mark_esque MrMatthewCFB Very well could be I k...  
9  Joe Biden cares more about the millions of pot...

In [13]:
sentiment_result = []
sentiment_result=sent_ana.predict(vec.transform(newdata["Word_Split"].values))
sentiment_result.shape
sentiment_result[:10]
newdata["sentiment"]=sentiment_result
newdata.head()
sentiment_otherform = []
for sent_num in newdata["sentiment"]:
    if (sent_num == 1):
        sentiment_otherform.append('Positive')
    elif (sent_num == 0):
        sentiment_otherform.append('Negative')
        
newdata["sentiment"]=sentiment_otherform
newdata.head()

created_at  \
1  2022-11-07T23:44:44.000Z   
3  2022-11-07T23:40:17.000Z   
5  2022-11-07T23:15:40.000Z   
7  2022-11-07T22:36:52.000Z   
9  2022-11-07T21:54:10.000Z   

                                                text          author.location  \
1  I am nervous but optimistic Democrats will win...           Huntsville, AL   
3  @joysephstjohn Brilliant interview with Kathy ...     Huntsville, Alabama    
5  @JoeBiden we need another stimulus check $$ ht...           Birmingham, AL   
7  @mark_esque @MrMatthewCFB Very well could be, ...  Mobile, Alabama/ Auburn   
9  Joe Biden cares more about the millions of pot...          Heart of Dixie    

     geo.country   geo.full_name    geo.name  \
1  United States  Huntsville, AL  Huntsville   
3  United States  Huntsville, AL  Huntsville   
5  United States  Birmingham, AL  Birmingham   
7  United States      Daphne, AL      Daphne   
9  United States     Satsuma, AL     Satsuma   

                                          Word_Split sentiment  
1  I am nervous but optimistic Democrats will win...  Positive  
3  joysephstjohn Brilliant interview with Kathy W...  Positive  
5  JoeBiden we need another stimulus check https ...  Positive  
7  mark_esque MrMatthewCFB Very well could be I k...  Negative  
9  Joe Biden cares more about the millions of pot...  Negative

In [14]:
data = pd.read_csv('C:/Users/User/new10k.csv')
newdata=data[["created_at","text","author.location","geo.country","geo.full_name","geo.name"]]
newdata.isnull().sum().sort_values(ascending=False)
newdata=newdata.dropna()
text10k=[]
for ttext in newdata["text"].apply(str):
    Word_Split = []
    for word in  re.sub("\W"," ",ttext ).split():
        Word_Split.append(word)
    text10k.append(' '.join(Word_Split))
newdata["Word_Split"]= text10k
newdata.head()

created_at  \
1  2022-11-07T23:44:44.000Z   
3  2022-11-07T23:40:17.000Z   
5  2022-11-07T23:15:40.000Z   
7  2022-11-07T22:36:52.000Z   
9  2022-11-07T21:54:10.000Z   

                                                text          author.location  \
1  I am nervous but optimistic Democrats will win...           Huntsville, AL   
3  @joysephstjohn Brilliant interview with Kathy ...     Huntsville, Alabama    
5  @JoeBiden we need another stimulus check $$ ht...           Birmingham, AL   
7  @mark_esque @MrMatthewCFB Very well could be, ...  Mobile, Alabama/ Auburn   
9  Joe Biden cares more about the millions of pot...          Heart of Dixie    

     geo.country   geo.full_name    geo.name  \
1  United States  Huntsville, AL  Huntsville   
3  United States  Huntsville, AL  Huntsville   
5  United States  Birmingham, AL  Birmingham   
7  United States      Daphne, AL      Daphne   
9  United States     Satsuma, AL     Satsuma   

                                          Word_Split  
1  I am nervous but optimistic Democrats will win...  
3  joysephstjohn Brilliant interview with Kathy W...  
5  JoeBiden we need another stimulus check https ...  
7  mark_esque MrMatthewCFB Very well could be I k...  
9  Joe Biden cares more about the millions of pot...

In [15]:
sentiment_result = []
for words in newdata["Word_Split"]:
    Tweet = TextBlob(words)
    sentiment_value = Tweet.sentiment[0]
    if (sentiment_value > 0):
        sentiment_result.append('Positive')
    elif (sentiment_value < 0):
        sentiment_result.append('Negative')
    else:
        sentiment_result.append('Neutral')

In [16]:
newdata["sentiment"]=sentiment_result
newdata.head()
newdata.to_csv('C:/Users/User/20k_sentiment11.csv')